# Importing all the important libraries

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install transformers sentence_transformers sentencepiece evaluate sacrebleu bert-score

In [ ]:
!conda update -n base -c conda-forge conda -y

In [ ]:
!conda install conda-forge::rouge-score -y

In [1]:
import pandas as pd
import tensorflow as tf
import torch
import ast
import numpy as np
import pprint
import sentencepiece as spm
import matplotlib.pyplot as plt
import torch.nn.functional as F
import bert_score
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformers import AutoTokenizer, AutoModelForMaskedLM, logging
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from torch.utils.data import DataLoader, Dataset
from rouge_score import rouge_scorer
from sacrebleu import corpus_chrf

<<<<<<< local
2025-02-05 00:37:34.798154: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-05 00:37:34.815300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-05 00:37:34.840247: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-05 00:37:34.840275: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-05 00:37:34.855632: I tensorflow/core/platform/c

# Set GPU

## Mac

In [2]:
# for mac
devices = tf.config.list_physical_devices()
print("\nDevices: ", devices)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
        details = tf.config.experimental.get_device_details(gpu)
        print("GPU details: ", details)
else:
    print("No GPU found. Using CPU.")

# set GPU device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")


Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU details:  {'device_name': 'METAL'}
Using device: mps


## Window / Linux

In [3]:
# for window
print("Tensorflow GPUs: ", tf.config.list_physical_devices('GPU'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using PyTorch device:", device)
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

Tensorflow GPUs:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Using PyTorch device: cuda
GPU Name: Tesla T4


<<<<<<< local
2025-02-05 00:37:42.127506: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-05 00:37:42.212323: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-05 00:37:42.213543: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs

In [3]:
# Load BLEU and ROUGE metric objects
bleu_metric = evaluate.load("bleu")
rouge_scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

# Class

In [3]:
# Custom dataset class for batching
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = [str(text) if text is not None else "" for text in texts] 

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

In [4]:
# custom dataset class for evaluation dataset
class EvaluationDataset(Dataset):
    def __init__(self, dataframe):
        self.predictions = dataframe["generated"].astype(str).tolist()
        self.references = dataframe["burmese"].astype(str).tolist()

    def __len__(self):
        return len(self.predictions)

    def __getitem__(self, idx):
        return self.predictions[idx], self.references[idx]

# Functions

In [5]:
# function to save models df
def save_models_df(df, df_name):
    df.to_csv(f"models/{df_name}.csv", index=False, encoding="utf-8")

In [6]:
# function to save tmp df
def save_tmp_df(df, df_name):
    df.to_csv(f"tmp/{df_name}.csv", index=False, encoding="utf-8")

In [7]:
# function to load spt df
def load_spt_df(df_name):
    return pd.read_csv(f"spt/{df_name}.csv", header=0, encoding="utf-8")

In [8]:
# function to load models df
def load_models_df(df_name):
    return pd.read_csv(f"models/{df_name}.csv", header=0, encoding="utf-8")

In [9]:
# function to load gen df
def load_gen_df(df_name):
    return pd.read_csv(f"gen/{df_name}.csv", header=0, encoding="utf-8")

In [10]:
# function to load tmp df
def load_tmp_df(df_name):
    return pd.read_csv(f"tmp/{df_name}.csv", header=0, encoding="utf-8")

In [11]:
# function to compute bleu score
def compute_bleu(reference, prediction):
    smoothing_fn = SmoothingFunction().method1
    return sentence_bleu([reference.split()], prediction.split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing_fn)

In [12]:
def safe_eval(val):
    return ast.literal_eval(val) if isinstance(val, str) else val

In [13]:
def compute_metrics_batch(dataset, batch_size=32):
    dataloader = DataLoader(EvaluationDataset(dataset), batch_size=batch_size, shuffle=False)

    all_bleu_scores, all_rouge1, all_rouge2, all_rougeL, all_chrfs, all_berts = [], [], [], [], [], []

    smooth_fn = SmoothingFunction().method1
    rouge = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    
    for batch in tqdm(dataloader, desc="Computing Metrics", unit="batch"):
        predictions, references = batch

        # Compute BLEU in batch
        batch_bleu = [sentence_bleu([ref.split()], pred.split(), smoothing_function=smooth_fn)
                      for pred, ref in zip(predictions, references)]
        all_bleu_scores.extend(batch_bleu)

        # Compute ROUGE in batch
        batch_rouge = [rouge.score(pred, ref) for pred, ref in zip(predictions, references)]
        all_rouge1.extend([r["rouge1"].fmeasure for r in batch_rouge])
        all_rouge2.extend([r["rouge2"].fmeasure for r in batch_rouge])
        all_rougeL.extend([r["rougeL"].fmeasure for r in batch_rouge])

        # Compute chrF-S in batch
        batch_chrf = corpus_chrf(predictions, [[ref] for ref in references]).score
        all_chrfs.extend([batch_chrf] * len(predictions))  # Apply same batch score to all

        # Compute BERTScore in batch
        batch_bert = bert_score.score(predictions, references, lang="my", device=device)
        all_berts.extend(batch_bert[2].tolist())  # F1 scores from BERTScore

    print("Finished Computing Metrics!")

    # Store results back into dataset
    dataset["bleu"] = all_bleu_scores
    dataset["rouge-1"] = all_rouge1
    dataset["rouge-2"] = all_rouge2
    dataset["rouge-l"] = all_rougeL
    dataset["chrf-s"] = all_chrfs
    dataset["bert_score"] = all_berts

In [14]:
# Function to compute perplexity in batch
def compute_perplexity_batch(texts, tokenizer, model):
    # Ensure all inputs are valid strings
    valid_texts = [str(text) if isinstance(text, str) else "" for text in texts]

    # Tokenize batch with padding & truncation
    inputs = tokenizer(valid_texts, return_tensors="pt", truncation=True, padding=True).to(device)

    # Get model outputs
    with torch.inference_mode():
        outputs = model(**inputs)

    logits = outputs.logits  # (batch_size, seq_len, vocab_size)
    log_probs = F.log_softmax(logits, dim=-1)  # Compute log probabilities

    # Get token log-likelihoods using true token IDs
    target_ids = inputs["input_ids"]  # (batch_size, seq_len)
    log_likelihood = log_probs.gather(dim=-1, index=target_ids.unsqueeze(-1)).squeeze(-1)

    # Apply attention mask to remove padding tokens
    mask = inputs["attention_mask"]  # (batch_size, seq_len)
    masked_log_likelihood = log_likelihood * mask  # Zero out padding contributions

    # Compute sentence-level mean log-likelihood (only over valid tokens)
    sentence_log_likelihood = masked_log_likelihood.sum(dim=1) / mask.sum(dim=1)

    # Clamp values to avoid numerical instability
    sentence_log_likelihood = torch.clamp(sentence_log_likelihood, min=-100, max=100)

    # Convert log-likelihood to perplexity
    log_perplexity = -sentence_log_likelihood
    perplexities = torch.exp(log_perplexity)

    # 🔍 Print warning if perplexities contain `inf`
    if torch.isinf(perplexities).any():
        print("\n [WARNING] Perplexity contains `inf` for some texts!")
        print(f"  Log-Likelihood Shape: {log_likelihood.shape}")
        print(f"  Log-Likelihood Mean: {sentence_log_likelihood.mean().item()}")
        print(f"  Computed Perplexity Values: {perplexities}")

    return perplexities.tolist()

# Set settings

In [15]:
tqdm.pandas()

In [16]:
# Suppress specific warnings from the transformers library
logging.set_verbosity_error()

# 1. Implementing RNN/LSTM Baseline

In [17]:
spt_models = {
    "bpe": spm.SentencePieceProcessor("spt/spt_bpe.model"),
    "unigram": spm.SentencePieceProcessor("spt/spt_unigram.model"),
}

## Data Preprocessing
Load SPT-tokenized datasets, convert to sequences, and apply padding.

### Load Data

In [18]:
# function to load datasets
def get_lstm_datasets(model_name):
    return {
        "normal": [
            f"tokenized_{model_name}_myxnli_normalized_1", 
            f"tokenized_{model_name}_myxnli_normalized_2", 
            f"tokenized_{model_name}_alt_combined_normalized"
        ],
        "nllb_back_translated": [
            f"tokenized_{model_name}_myxnli_nllb_back_translated_final_1", 
            f"tokenized_{model_name}_myxnli_nllb_back_translated_final_2", 
            f"tokenized_{model_name}_alt_combined_nllb_back_translated_final"
        ],
        "seamless_m4t_back_translated": [
            f"tokenized_{model_name}_myxnli_seamless_m4t_back_translated_final_1",
            f"tokenized_{model_name}_myxnli_seamless_m4t_back_translated_final_2",
            f"tokenized_{model_name}_alt_combined_seamless_m4t_back_translated_final"
        ],
    }

In [19]:
# Load and process dataset
def load_and_rename_columns_lstm(file_name):
    df = load_spt_df(f"{file_name}")

    column_mapping = {
        "english_back_translated": "english",
        "burmese_translated": "burmese",
        "english_back_translated_tokens": "english_tokens",
        "burmese_translated_tokens": "burmese_tokens",
    }
    
    df = df.rename(columns=column_mapping)
    
    # Ensure only required columns exist
    df = df[["english", "burmese", "english_tokens", "burmese_tokens"]]

    return df

In [20]:
# Load all datasets
lstm_all_datasets = {}
for model_name in spt_models.keys():
    datasets = get_lstm_datasets(model_name)

    lstm_all_datasets[model_name] = {
        key: [load_and_rename_columns_lstm(file) for file in file_list] for key, file_list in datasets.items()
    }

In [21]:
# cobine all datasets
lstm_all_datasets_combined = {}
for model_name in lstm_all_datasets.keys():
    lstm_all_datasets_combined[model_name] = pd.concat(
        [pd.concat(datasets) for datasets in lstm_all_datasets[model_name].values()],
        ignore_index=True
    )

In [22]:
# Shuffle the data to prevent order bias and drop null
for model_name in lstm_all_datasets_combined.keys():
    lstm_all_datasets_combined[model_name] = lstm_all_datasets_combined[model_name].sample(frac=1).reset_index(drop=True)

In [23]:
# display of datasets
for model_name in lstm_all_datasets_combined.keys():
    print(f"{model_name} dataset length: {len(lstm_all_datasets_combined[model_name])}")

bpe dataset length: 1627576
unigram dataset length: 1627576


### Apply padding

In [16]:
# convert tokenized sequences to lists
for model_name in lstm_all_datasets_combined.keys():
    lstm_all_datasets_combined[model_name]["english_seq"] = lstm_all_datasets_combined[model_name]["english_tokens"].progress_apply(
        lambda x: spt_models[model_name].EncodeAsIds(str(x)) if isinstance(x, str) else []
    )
    lstm_all_datasets_combined[model_name]["burmese_seq"] = lstm_all_datasets_combined[model_name]["burmese_tokens"].progress_apply(
        lambda x:  spt_models[model_name].EncodeAsIds(str(x)) if isinstance(x, str) else []
    )

  0%|          | 0/1627576 [00:00<?, ?it/s]

  0%|          | 0/1627576 [00:00<?, ?it/s]

  0%|          | 0/1627576 [00:00<?, ?it/s]

  0%|          | 0/1627576 [00:00<?, ?it/s]

In [18]:
# Define maximum sequence length
lstm_max_seq_length = 128

In [19]:
# appply padding to sequences
for model_name in lstm_all_datasets_combined.keys():
    lstm_all_datasets_combined[model_name]["english_seq_padded"] = pad_sequences(
        lstm_all_datasets_combined[model_name]["english_seq"], maxlen=lstm_max_seq_length, padding="post"
    ).tolist()

    lstm_all_datasets_combined[model_name]["burmese_seq_padded"] = pad_sequences(
        lstm_all_datasets_combined[model_name]["burmese_seq"], maxlen=lstm_max_seq_length, padding="post"
    ).tolist()

In [20]:
# save lstm preprocess data
for model_name in lstm_all_datasets_combined.keys():
    save_models_df(lstm_all_datasets_combined[model_name], f"lstm_{model_name}_preprocessed")

## Define LSTM Model
Define an LSTM-based sequence-to-sequence (seq2seq) model with embedding layers.

In [18]:
# Hyperparameters
lstm_embedding_dim = 256
lstm_hidden_dim = 512

In [19]:
# Get vocabulary size from SentencePiece models
lstm_vocab_sizes = {model_name: sp.GetPieceSize() for model_name, sp in spt_models.items()}

In [20]:
# function to build lstm model
def build_lstm_model(vocab_size):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=lstm_embedding_dim, mask_zero=True),
        Bidirectional(LSTM(lstm_hidden_dim, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)),
        LSTM(lstm_hidden_dim, return_sequences=True, dropout=0.3, recurrent_dropout=0.3),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [21]:
# build lstm bpe model
lstm_bpe_model = build_lstm_model(lstm_vocab_sizes["bpe"])
lstm_bpe_model.summary()

2025-02-03 08:15:44.588870: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-03 08:15:44.591696: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-03 08:15:44.593767: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         8192000   
                                                                 
 bidirectional (Bidirection  (None, None, 1024)        3149824   
 al)                                                             
                                                                 
 lstm_1 (LSTM)               (None, None, 512)         3147776   
                                                                 
 dense (Dense)               (None, None, 32000)       16416000  
                                                                 
Total params: 30905600 (117.90 MB)
Trainable params: 30905600 (117.90 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
# build lstm bpe model
lstm_unigram_model = build_lstm_model(lstm_vocab_sizes["unigram"])
lstm_unigram_model.summary()

2025-02-03 08:36:43.437264: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-03 08:36:43.512506: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-03 08:36:43.515652: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         8192000   
                                                                 
 bidirectional (Bidirection  (None, None, 1024)        3149824   
 al)                                                             
                                                                 
 lstm_1 (LSTM)               (None, None, 512)         3147776   
                                                                 
 dense (Dense)               (None, None, 32000)       16416000  
                                                                 
Total params: 30905600 (117.90 MB)
Trainable params: 30905600 (117.90 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Train the Model
Train the model using Categorical Cross-Entropy loss & Adam optimizer.

In [18]:
# lstm model train batch size
lstm_train_batch_size = 64
lstm_train_epochs = 2

### BPE

In [19]:
# model prefix
lstm_bpe_model_prefix = "models/lstm_bpe_model"

In [20]:
# load lstm bpe dataset
lstm_bpe_preprocessed = load_models_df("lstm_bpe_preprocessed")

In [21]:
# Convert from string to list
lstm_bpe_preprocessed["burmese_seq_padded"] = lstm_bpe_preprocessed["burmese_seq_padded"].apply(safe_eval)
lstm_bpe_preprocessed["english_seq_padded"] = lstm_bpe_preprocessed["english_seq_padded"].apply(safe_eval)

In [22]:
# Convert to NumPy arrays
lstm_bpe_X_train = np.array(lstm_bpe_preprocessed["burmese_seq_padded"].tolist(), dtype=np.int32)
lstm_bpe_y_train = np.array(lstm_bpe_preprocessed["english_seq_padded"].tolist(), dtype=np.int32)

print(f"X_train shape: {lstm_bpe_X_train.shape}")
print(f"y_train shape: {lstm_bpe_y_train.shape}")

X_train shape: (1627576, 128)
y_train shape: (1627576, 128)


In [23]:
# Callbacks: Early Stopping + Model Checkpoint
lstm_bpe_early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lstm_bpe_checkpoint = ModelCheckpoint(
    f"{lstm_bpe_model_prefix}.keras", 
    save_best_only=True, 
    monitor='val_loss', 
    mode='min',
    save_weights_only=False
)

In [ ]:
# Train model
lstm_bpe_model.fit(
    lstm_bpe_X_train, 
    lstm_bpe_y_train, 
    batch_size=lstm_train_batch_size,
    epochs=lstm_train_epochs, 
    validation_split=0.1, 
    callbacks=[lstm_bpe_early_stopping, lstm_bpe_checkpoint]
)

Epoch 1/5


I0000 00:00:1738618633.745274   25622 service.cc:145] XLA service 0x7f532af170a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1738618633.745310   25622 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-02-03 21:37:14.172679: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-03 21:37:14.804654: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1738618635.464858   25622 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


22888/22888 [==============================] - 28114s 1s/step - loss: 1.5701 - accuracy: 0.6595 - val_loss: 1.6427 - val_accuracy: 0.6395
Epoch 2/5
22888/22888 [==============================] - 27965s 1s/step - loss: 1.5478 - accuracy: 0.6636 - val_loss: 1.6377 - val_accuracy: 0.6406
Epoch 3/5
22888/22888 [==============================] - 27752s 1s/step - loss: 1.5296 - accuracy: 0.6669 - val_loss: 1.6369 - val_accuracy: 0.6408
Epoch 4/5
  385/22888 [..............................] - ETA: 7:27:51 - loss: 1.4893 - accuracy: 0.6770

In [ ]:
# Train model
lstm_bpe_model.fit(
    lstm_bpe_X_train, 
    lstm_bpe_y_train, 
    batch_size=lstm_train_batch_size,
    epochs=lstm_train_epochs, 
    validation_split=0.1, 
    callbacks=[lstm_bpe_early_stopping, lstm_bpe_checkpoint]
)

Epoch 1/2


I0000 00:00:1738716574.969466    5772 service.cc:145] XLA service 0x7f6fcc016ef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1738716574.969504    5772 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-02-05 00:49:34.976221: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-05 00:49:35.011981: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1738716575.075759    5772 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


22888/22888 [==============================] - 28155s 1s/step - loss: 1.5143 - accuracy: 0.6699 - val_loss: 1.6157 - val_accuracy: 0.6414
Epoch 2/2
22888/22888 [==============================] - 28097s 1s/step - loss: 1.5006 - accuracy: 0.6724 - val_loss: 1.6336 - val_accuracy: 0.6406


In [26]:
# save lstm bpe model
lstm_bpe_model.save(f"{lstm_bpe_model_prefix}.keras", save_format="keras")

### Unigram

In [19]:
# model prefix
lstm_unigram_model_prefix = "models/lstm_unigram_model"

In [20]:
# load lstm unigram dataset
lstm_unigram_preprocessed = load_models_df("lstm_unigram_preprocessed")

In [21]:
# Convert from string to list
lstm_unigram_preprocessed["burmese_seq_padded"] = lstm_unigram_preprocessed["burmese_seq_padded"].apply(safe_eval)
lstm_unigram_preprocessed["english_seq_padded"] = lstm_unigram_preprocessed["english_seq_padded"].apply(safe_eval)

In [22]:
# Convert to NumPy arrays
lstm_unigram_X_train = np.array(lstm_unigram_preprocessed["burmese_seq_padded"].tolist(), dtype=np.int32)
lstm_unigram_y_train = np.array(lstm_unigram_preprocessed["english_seq_padded"].tolist(), dtype=np.int32)

print(f"X_train shape: {lstm_unigram_X_train.shape}")
print(f"y_train shape: {lstm_unigram_y_train.shape}")

X_train shape: (1627576, 128)
y_train shape: (1627576, 128)


In [23]:
# Callbacks: Early Stopping + Model Checkpoint
lstm_unigram_early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lstm_unigram_checkpoint = ModelCheckpoint(
    f"{lstm_unigram_model_prefix}.keras", 
    save_best_only=True, 
    monitor='val_loss', 
    mode='min',
    save_weights_only=False
)

In [ ]:
# Train model
lstm_unigram_model.fit(
    lstm_unigram_X_train, 
    lstm_unigram_y_train, 
    batch_size=lstm_train_batch_size,
    epochs=lstm_train_epochs, 
    validation_split=0.1, 
    callbacks=[lstm_unigram_early_stopping, lstm_unigram_checkpoint]
)

Epoch 1/5


I0000 00:00:1738618686.064670   25952 service.cc:145] XLA service 0x7f83f2cc1640 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1738618686.064704   25952 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-02-03 21:38:06.288576: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-03 21:38:06.670883: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1738618687.125419   25952 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


22888/22888 [==============================] - 27693s 1s/step - loss: 1.2076 - accuracy: 0.7599 - val_loss: 1.2439 - val_accuracy: 0.7482
Epoch 2/5
22888/22888 [==============================] - 27550s 1s/step - loss: 1.1834 - accuracy: 0.7633 - val_loss: 1.2398 - val_accuracy: 0.7481
Epoch 3/5
22888/22888 [==============================] - 27326s 1s/step - loss: 1.1643 - accuracy: 0.7659 - val_loss: 1.2261 - val_accuracy: 0.7483
Epoch 4/5
 1697/22888 [=>............................] - ETA: 6:49:34 - loss: 1.1317 - accuracy: 0.7720

In [ ]:
# Train model
lstm_unigram_model.fit(
    lstm_unigram_X_train,
    lstm_unigram_y_train,
    batch_size=lstm_train_batch_size,
    epochs=lstm_train_epochs,
    validation_split=0.1,
    callbacks=[lstm_unigram_early_stopping, lstm_unigram_checkpoint]
)

Epoch 1/2


I0000 00:00:1738716422.095172   15580 service.cc:145] XLA service 0x7f285f2d0a20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1738716422.095209   15580 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-02-05 00:47:02.101910: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-05 00:47:02.121342: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1738716422.182801   15580 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


22888/22888 [==============================] - 27650s 1s/step - loss: 1.1481 - accuracy: 0.7684 - val_loss: 1.2335 - val_accuracy: 0.7479
Epoch 2/2
22888/22888 [==============================] - 27903s 1s/step - loss: 1.1344 - accuracy: 0.7703 - val_loss: 1.2331 - val_accuracy: 0.7475


In [27]:
# save lstm bpe model
lstm_unigram_model.save(f"{lstm_unigram_model_prefix}.keras", save_format="keras")

## Generate Predictions
Use trained LSTM models to generate translations for evaluation.

In [ ]:
# LSTM Text Dataset
class LstmTextDataset(Dataset):
    """Custom dataset for text sequences"""
    def __init__(self, texts, tokenizer, max_length=50):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        tokenized_text = self.tokenizer.encode(text, out_type=int)

        # Pad sequences to max_length
        tokenized_text = tokenized_text[:self.max_length]
        padding_length = self.max_length - len(tokenized_text)
        padded_sequence = tokenized_text + [0] * padding_length  # Padding with 0

        return torch.tensor(padded_sequence, dtype=torch.long)

In [ ]:
# function to generate text using LSTM in auto-regressive manner for batch processing.
def generate_text_lstm_batch(dataloader, model, tokenizer, max_length=50):
    all_predictions = []

    for batch in tqdm(dataloader, desc="Generating Text with LSTM"):
        batch = batch.numpy()  # Convert to NumPy for TensorFlow processing
        batch_generated = []

        for seq in batch:
            generated_sequence = seq.tolist()

            for _ in range(max_length - len(seq)):
                input_padded = pad_sequences(
                    [generated_sequence], maxlen=max_length, padding='pre'
                )
                input_tensor = tf.convert_to_tensor(input_padded, dtype=tf.int32)

                predictions = model.predict(input_tensor, verbose=0)
                next_token = np.argmax(predictions[0], axis=-1)

                if next_token == tokenizer.eos_id():
                    break  # Stop at EOS token

                generated_sequence.append(next_token)

            batch_generated.append(tokenizer.decode(generated_sequence))

        all_predictions.extend(batch_generated)

    return all_predictions

### BPE

In [ ]:
# load lstm bpe model and tokenizer
lstm_bpe_model = load_model(f"models/lstm_bpe_model.h5")
lstm_bpe_tokenizer = spt_models["bpe"]

In [ ]:
# load lstm bpe dataset
lstm_bpe_predictions = load_models_df("lstm_bpe_preprocessed")

In [ ]:
# Convert from string to list and NumPy arrays
lstm_bpe_predictions["burmese_seq_padded"] = lstm_bpe_predictions["burmese_seq_padded"].apply(safe_eval)
lstm_bpe_predictions_sequences = np.array(lstm_bpe_predictions["burmese_seq_padded"].tolist(), dtype=np.int32)

In [ ]:
# batch size
lstm_bpe_predictions_batch_size = 8

In [ ]:
# Convert to DataLoader
lstm_bpe_predictions_dataset = LstmTextDataset(lstm_bpe_predictions_sequences, lstm_bpe_tokenizer)
lstm_bpe_predictions_dataloader = DataLoader(lstm_bpe_predictions_dataset, batch_size=lstm_bpe_predictions_batch_size, shuffle=False)

In [ ]:
# Run text generation
lstm_bpe_predictions["generated"] = generate_text_lstm_batch(lstm_bpe_predictions_dataloader, lstm_bpe_model, lstm_bpe_tokenizer)

In [ ]:
# remove some columns and display
lstm_bpe_predictions = lstm_bpe_predictions[lstm_bpe_predictions["english", "burmese", "generated"]]
display(lstm_bpe_predictions)

In [ ]:
# save lstm bpe predictions
save_models_df(lstm_bpe_predictions, "lstm_bpe_predictions")

### Unigram

In [ ]:
# load lstm unigram model and tokenizer
lstm_unigram_model = load_model(f"models/lstm_unigram_model.h5")
lstm_unigram_tokenizer = spt_models["unigram"]

In [ ]:
# load lstm unigram dataset
lstm_unigram_predictions = load_models_df("lstm_unigram_preprocessed")

In [ ]:
# Convert from string to list and NumPy arrays
lstm_unigram_predictions["burmese_seq_padded"] = lstm_unigram_predictions["burmese_seq_padded"].apply(safe_eval)
lstm_unigram_predictions_sequences = np.array(lstm_unigram_predictions["burmese_seq_padded"].tolist(), dtype=np.int32)

In [ ]:
# batch size
lstm_unigram_predictions_batch_size = 8

In [ ]:
# Convert to DataLoader
lstm_unigram_predictions_dataset = LstmTextDataset(lstm_unigram_predictions_sequences, lstm_unigram_tokenizer)
lstm_unigram_predictions_dataloader = DataLoader(lstm_unigram_predictions_dataset, batch_size=lstm_unigram_predictions_batch_size, shuffle=False)

In [ ]:
# Run text generation
lstm_unigram_predictions["generated"] = generate_text_lstm_batch(lstm_unigram_predictions_dataloader, lstm_unigram_model, lstm_unigram_tokenizer)

In [ ]:
# remove some columns and display
lstm_unigram_predictions = lstm_unigram_predictions[lstm_unigram_predictions["english", "burmese", "generated"]]
display(lstm_unigram_predictions)

In [ ]:
# save lstm unigram predictions
save_models_df(lstm_unigram_predictions, "lstm_unigram_predictions")

## Evaluate Model Performance
Compute BLEU, ROUGE-1, ROUGE-2, ROUGE-3, ROUGE-L, chrF-S, BERTScore and Perplexity scores.

In [ ]:
# load lstm predictions
lstm_evaluation_results_datasets = {
    model_name: load_models_df(f"lstm_{model_name}_predictions") for model_name in spt_models.keys()
}

### Metrics

In [ ]:
# compute metrics
for model_name, dataset in lstm_evaluation_results_datasets.items():
    print(f"Processing Data for {model_name}...")
    compute_metrics_batch(dataset)

In [ ]:
# display metrics
for model_name, dataset in lstm_evaluation_results_datasets.items():
    print(f"Metrics scores for {model_name}:")
    print(f" BLEU Score: {dataset['bleu'].mean()}")
    print(f" ROUGE-1 Score: {dataset['rouge-1'].mean()}")
    print(f" ROUGE-2 Score: {dataset['rouge-2'].mean()}")
    print(f" ROUGE-L Score: {dataset['rouge-l'].mean()}")
    print(f" chrF-S Score: {dataset['chrf-s'].mean()}")
    print(f" BERT Score: {dataset['bert'].mean()}")

In [ ]:
# save resutls
for model_name, dataset in lstm_evaluation_results_datasets.items():
    save_tmp_df(dataset, f"lstm_{model_name}_metrics")

### Perplexity

#### BPE

In [ ]:
# batch size
lstm_bpe_perplexity_batch_size = 8

In [ ]:
# load lstm bpe model and tokenizer
lstm_bpe_model = load_model(f"models/lstm_bpe_model.h5")
lstm_bpe_tokenizer = spt_models["bpe"]

In [ ]:
# Prepare dataset and DataLoader
lstm_bpe_generated_texts = lstm_evaluation_results_datasets["bpe"]["generated"].tolist()
lstm_bpe_text_dataset = TextDataset(lstm_bpe_generated_texts)
lstm_bpe_dataloader = DataLoader(
    lstm_bpe_text_dataset, 
    batch_size=lstm_bpe_perplexity_batch_size, 
    shuffle=False
)

In [ ]:
# Compute perplexity in batches
lstm_bpe_perplexity_scores = []
for batch in tqdm(lstm_bpe_dataloader, desc="Computing Perplexity for LSTM BPE"):
    batch_perplexities = compute_perplexity_batch(batch, lstm_bpe_tokenizer, lstm_bpe_model)
    lstm_bpe_perplexity_scores.extend(batch_perplexities)

In [ ]:
# Store perplexity scores in DataFrame and display
lstm_evaluation_results_datasets["bpe"]["perplexity"] = lstm_bpe_perplexity_scores
print(f"Perplexity Score: {lstm_evaluation_results_datasets['bpe']['perplexity'].mean()}")

In [ ]:
# save perplexity
save_tmp_df(lstm_evaluation_results_datasets["bpe"], f"lstm_bpe_perplexity")

#### Unigram

In [ ]:
# batch size
lstm_unigram_perplexity_batch_size = 8

In [ ]:
# load lstm unigram model and tokenizer
lstm_unigram_model = load_model(f"models/lstm_unigram_model.h5")
lstm_unigram_tokenizer = spt_models["unigram"]

In [ ]:
# Prepare dataset and DataLoader
lstm_unigram_generated_texts = lstm_evaluation_results_datasets["unigram"]["generated"].tolist()
lstm_unigram_text_dataset = TextDataset(lstm_unigram_generated_texts)
lstm_unigram_dataloader = DataLoader(
    lstm_unigram_text_dataset, 
    batch_size=lstm_unigram_perplexity_batch_size, 
    shuffle=False
)

In [ ]:
# Compute perplexity in batches
lstm_unigram_perplexity_scores = []
for batch in tqdm(lstm_unigram_dataloader, desc="Computing Perplexity for LSTM Unigram"):
    batch_perplexities = compute_perplexity_batch(batch, lstm_unigram_tokenizer, lstm_unigram_model)
    lstm_unigram_perplexity_scores.extend(batch_perplexities)

In [ ]:
# Store perplexity scores in DataFrame and display
lstm_evaluation_results_datasets["unigram"]["perplexity"] = lstm_unigram_perplexity_scores
print(f"Perplexity Score: {lstm_evaluation_results_datasets['unigram']['perplexity'].mean()}")

In [ ]:
# save perplexity
save_tmp_df(lstm_evaluation_results_datasets["unigram"], f"lstm_unigram_perplexity")

### Save Evaluation Results

In [ ]:
# combine evaluation results
for model_name in lstm_evaluation_results_datasets.keys():
    print(f"Processing {model_name}...")

    # load metrics and set
    metrics = load_tmp_df(f"lstm_{model_name}_metrics")
    lstm_evaluation_results_datasets[model_name]["bleu"] = metrics["bleu"]
    lstm_evaluation_results_datasets[model_name]["rouge-1"] = metrics["rouge-1"]
    lstm_evaluation_results_datasets[model_name]["rouge-2"] = metrics["rouge-2"]
    lstm_evaluation_results_datasets[model_name]["rouge-l"] = metrics["rouge-l"]
    lstm_evaluation_results_datasets[model_name]["chrf-s"] = metrics["chrf-s"]
    lstm_evaluation_results_datasets[model_name]["bert_score"] = metrics["bert_score"]

    # load perplexity and set
    perplexity = load_tmp_df(f"lstm_{model_name}_perplexity")
    lstm_evaluation_results_datasets[model_name]["perplexity"] = perplexity["perplexity"]

    save_models_df(lstm_evaluation_results_datasets[model_name], f"lstm_{model_name}_evaluation_results")

    display(lstm_evaluation_results_datasets[model_name].head())

# 2. Implementing Multilingual Transformer Baseline

In [17]:
# Define model names
multilingual_model_names = {
    "mbert": "bert-base-multilingual-cased",
    "xlmr": "xlm-roberta-base"
}

## Data Preprocessing

In [18]:
# datasets
multilingual_datasets = {
    "normal": [
        "myxnli_normalized_1", 
        "myxnli_normalized_2", 
        "alt_combined_normalized"
    ],
    "nllb_back_translated": [
        "myxnli_nllb_back_translated_final_1", 
        "myxnli_nllb_back_translated_final_2", 
        "alt_combined_nllb_back_translated_final"
    ],
    "seamless_m4t_back_translated": [
        "myxnli_seamless_m4t_back_translated_final_1",
        "myxnli_seamless_m4t_back_translated_final_2",
        "alt_combined_seamless_m4t_back_translated_final"
    ],
}

In [11]:
# Load and process dataset
def load_and_rename_columns_multilingual(file_name):
    df = load_gen_df(f"{file_name}")

    column_mapping = {
        "english_back_translated": "english",
        "burmese_translated": "burmese",
    }
    
    df = df.rename(columns=column_mapping)
    
    # Ensure only required columns exist
    df = df[["english", "burmese"]]

    return df

In [12]:
# Load and process datasets
mutlilingual_loaded_datasets = {}
for key, file_list in multilingual_datasets.items():
    mutlilingual_loaded_datasets[key] = [load_and_rename_columns_multilingual(file) for file in file_list]

In [13]:
# combine all datasets
multilingual_combined = pd.concat(
    mutlilingual_loaded_datasets["normal"] + 
    mutlilingual_loaded_datasets["nllb_back_translated"] + 
    mutlilingual_loaded_datasets["seamless_m4t_back_translated"],
    ignore_index=True
)

In [14]:
# Shuffle the data to prevent order bias
multilingual_combined = multilingual_combined.sample(frac=1).reset_index(drop=True)

In [16]:
# print length
print(f"Multilingual dataset length: {len(multilingual_combined)}")

Multilingual dataset length: 1627576


In [17]:
# save data
save_models_df(multilingual_combined, "multilingual_combined")

## Generate Predictions
Load ```mBERT``` and ```XLM-R``` for Masked Language Modeling (MLM).
MLM helps predict missing words in Burmese sequences.

In [18]:
# dataset class for masked
class MaskedTextDataset(Dataset):
    def __init__(self, texts, tokenizer, mask_ratio=0.15, max_length=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.mask_ratio = mask_ratio
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx] if isinstance(self.texts[idx], str) else ""

        # Tokenize and move tensors to GPU
        inputs = self.tokenizer(
            text, return_tensors="pt", truncation=True, padding="max_length", max_length=self.max_length
        )
        
        input_ids = inputs["input_ids"].squeeze(0).to(device)
        attention_mask = inputs["attention_mask"].squeeze(0).to(device)

        # Apply random masking
        seq_length = input_ids.shape[0]
        num_to_mask = max(1, int(self.mask_ratio * (seq_length - 2)))  # Avoid CLS/SEP
        mask_indices = torch.randperm(seq_length - 2)[:num_to_mask] + 1  # Avoid first and last token

        masked_input_ids = input_ids.clone()
        masked_input_ids[mask_indices] = self.tokenizer.mask_token_id  # Replace with [MASK] token

        return masked_input_ids, attention_mask, input_ids

In [19]:
# Function to generate masked predictions
def generate_masked_predictions_batch(dataloader, model, tokenizer):
    all_predictions = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Generating Masked Predictions"):
            # Move batch data to GPU
            masked_input_ids, attention_mask, original_input_ids = [x.to(device) for x in batch]

            # Run model inference on GPU
            outputs = model(input_ids=masked_input_ids, attention_mask=attention_mask)

            # Replace masked tokens with predicted tokens
            predicted_tokens_batch = masked_input_ids.clone()
            for i in range(masked_input_ids.shape[0]):  # Loop over batch
                mask_positions = (masked_input_ids[i] == tokenizer.mask_token_id).nonzero(as_tuple=True)[0]
                for pos in mask_positions:
                    predicted_token_id = torch.argmax(outputs.logits[i, pos], dim=-1).item()
                    predicted_tokens_batch[i, pos] = predicted_token_id

            # Decode predictions
            batch_predictions = tokenizer.batch_decode(predicted_tokens_batch.cpu(), skip_special_tokens=True)
            all_predictions.extend(batch_predictions)

    return all_predictions

### mBert

In [20]:
# Load tokenizers & models for both mBERT
multilingual_mbert_tokenizer = AutoTokenizer.from_pretrained(multilingual_model_names["mbert"])
multilingual_mbert_model = AutoModelForMaskedLM.from_pretrained(multilingual_model_names["mbert"]).to(device)
multilingual_mbert_model = torch.compile(multilingual_mbert_model)
multilingual_mbert_model.eval()

OptimizedModule(
  (_orig_mod): BertForMaskedLM(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bia

In [21]:
# load multilingual dataset
multilingual_mbert_predictions = load_models_df("multilingual_combined")

In [22]:
# batch size
multilingual_mbert_predictions_batch_size = 8

In [23]:
# Convert to DataLoader
multilingual_mbert_predictions_texts = multilingual_mbert_predictions["burmese"].tolist()
multilingual_mbert_predictions_dataset = MaskedTextDataset(multilingual_mbert_predictions_texts, multilingual_mbert_tokenizer)
multilingual_mbert_predictions_dataloader = DataLoader(
    multilingual_mbert_predictions_dataset, 
    batch_size=multilingual_mbert_predictions_batch_size, 
    shuffle=False
)

In [24]:
# Run text generation
multilingual_mbert_predictions["generated"] = generate_masked_predictions_batch(
    multilingual_mbert_predictions_dataloader, 
    multilingual_mbert_model, 
    multilingual_mbert_tokenizer
)

Generating Masked Predictions:   0%|          | 0/203447 [00:00<?, ?it/s]

In [25]:
# display
display(multilingual_mbert_predictions.head())

,english,burmese,generated
0,it's not worth seeing the nubian floor exhibit...,အထက် အီဂျစ်မှာ နူဘီးယား ကြမ်းပြင်ပြပွဲကို ကြည့...,အထက် အီဂျစ်မှာ နူဘီးယား ကြမ်းပြင်ပြပွဲကို ကြည့...
1,there are remote whitewashed villages that adv...,စွန့်စားချင်သူတွေ လည်ပတ်ချင်ကြတဲ့ ဝေးလံခေါင်သီ...,စွန့်စားချင်သီတွေ လည်ပတ်ချင်ကြတဲ့ ဝေးလံခေါင်သီ...
2,"she makes these little tricks, very good, and ...",သူမက ဒီပျဉ်းစေ့ကြိုးတွေ လုပ်ပေးတယ် အရမ်းကောင်း...,သူမက စီပျဉ်းစေ့ကြိုးတွေ လုပ်ပေးတဲ့ ၊ိမ်းကောင်း...
3,the pair regained zimbabwe's times and finishe...,ထိုစုံတွဲသည် ဇင်ဘာဘွေ၏ အကြိမ်များကို ပြန်လည်ရရ...,ထိုစုံပွဲသည် ဇင်ဘာတွေ ၏ အကြိမ်များကို ပြန်လည်ရ...
4,potential of clarifying its notices to taxpaye...,အခွန်ထမ်းများထံ ၎င်း၏သတိပေးချက်များကို ရှင်းလင...,အခွက်ထမ်းများထံ ၎ င်း ၏ သတိပြချက်များကို ရှင်း...


In [26]:
# save multilingual mbert predictions
save_models_df(multilingual_mbert_predictions, "multilingual_mbert_predictions")

### XLM-R

In [19]:
# Load tokenizers & models for both xlmr
multilingual_xlmr_tokenizer = AutoTokenizer.from_pretrained(multilingual_model_names["xlmr"])
multilingual_xlmr_model = AutoModelForMaskedLM.from_pretrained(multilingual_model_names["xlmr"]).to(device)
multilingual_xlmr_model = torch.compile(multilingual_xlmr_model)
multilingual_xlmr_model.eval()

OptimizedModule(
  (_orig_mod): XLMRobertaForMaskedLM(
    (roberta): XLMRobertaModel(
      (embeddings): XLMRobertaEmbeddings(
        (word_embeddings): Embedding(250002, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): XLMRobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x XLMRobertaLayer(
            (attention): XLMRobertaAttention(
              (self): XLMRobertaSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): XLMRobertaSelfOutput(
    

In [20]:
# load multilingual dataset
multilingual_xlmr_predictions = load_models_df("multilingual_combined")

In [21]:
# multinlingual batch size
multilingual_xlmr_predictions_batch_size = 8

In [ ]:
# Convert to DataLoader
multilingual_xlmr_predictions_texts = multilingual_xlmr_predictions["burmese"].tolist()
multilingual_xlmr_predictions_dataset = MaskedTextDataset(multilingual_xlmr_predictions_texts, multilingual_xlmr_tokenizer)
multilingual_xlmr_predictions_dataloader = DataLoader(
    multilingual_xlmr_predictions_dataset, 
    batch_size=multilingual_xlmr_predictions_batch_size, 
    shuffle=False
)

In [ ]:
# Run text generation
multilingual_xlmr_predictions["generated"] = generate_masked_predictions_batch(
    multilingual_xlmr_predictions_dataloader, 
    multilingual_xlmr_model, 
    multilingual_xlmr_tokenizer
)

In [ ]:
# display
display(multilingual_xlmr_predictions.head())

In [19]:
# save multilingual xlmr predictions
save_models_df(multilingual_xlmr_predictions, "multilingual_xlmr_predictions")

## Evaluate Model Performance
Compute BLEU, ROUGE-1, ROUGE-2, ROUGE-3, ROUGE-L, chrF-S, BERTScore and Perplexity scores.

In [18]:
# load multilingual predictions
multilingual_evaluation_results_datasets = {
    model_name: load_models_df(f"multilingual_{model_name}_predictions") for model_name in multilingual_model_names.keys()
}

### Metrics

In [19]:
# compute metrics
for model_name, dataset in multilingual_evaluation_results_datasets.items():
    print(f"Processing Data for {model_name}..."),
    compute_metrics_batch(dataset)

Processing Data for mbert...


Computing Metrics:   0%|          | 0/50862 [00:00<?, ?batch/s]

Finished Computing Metrics!
Processing Data for xlmr...


Computing Metrics:   0%|          | 0/50862 [00:00<?, ?batch/s]

Finished Computing Metrics!


In [21]:
# display metrics
for model_name, dataset in multilingual_evaluation_results_datasets.items():
    print(f"Metrics scores for {model_name}:")
    print(f" BLEU Score: {dataset['bleu'].mean()}")
    print(f" ROUGE-1 Score: {dataset['rouge-1'].mean()}")
    print(f" ROUGE-2 Score: {dataset['rouge-2'].mean()}")
    print(f" ROUGE-L Score: {dataset['rouge-l'].mean()}")
    print(f" chrF-S Score: {dataset['chrf-s'].mean()}")
    print(f" BERT Score: {dataset['bert_score'].mean()}")

Metrics scores for mbert:
 BLEU Score: 0.11161891030998354
 ROUGE-1 Score: 0.21893887065417755
 ROUGE-2 Score: 0.0979642909372018
 ROUGE-L Score: 0.2188918748703697
 chrF-S Score: 79.22023133392412
 BERT Score: 0.8790649549057732
Metrics scores for xlmr:
 BLEU Score: 0.4581550312737298
 ROUGE-1 Score: 0.25353074248638974
 ROUGE-2 Score: 0.11654468643909702
 ROUGE-L Score: 0.2535243519409113
 chrF-S Score: 88.05020715366919
 BERT Score: 0.9653381555889077


In [22]:
# save results
for model_name, dataset in multilingual_evaluation_results_datasets.items():
    save_tmp_df(dataset, f"multilingual_{model_name}_metrics")

### Perplexity

#### mBert

In [19]:
# batch size
multilingual_mbert_perplexity_batch_size = 8

In [20]:
# Load tokenizers & models for both mbert
multilingual_mbert_tokenizer = AutoTokenizer.from_pretrained(multilingual_model_names["mbert"])
multilingual_mbert_model = AutoModelForMaskedLM.from_pretrained(multilingual_model_names["mbert"]).to(device)
multilingual_mbert_model = torch.compile(multilingual_mbert_model)
multilingual_mbert_model.eval()

OptimizedModule(
  (_orig_mod): BertForMaskedLM(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bia

In [21]:
# Prepare dataset and DataLoader
multilingual_mbert_generated_texts = multilingual_evaluation_results_datasets["mbert"]["generated"].tolist()
multilingual_mbert_text_dataset = TextDataset(multilingual_mbert_generated_texts)
multilingual_mbert_dataloader = DataLoader(
    multilingual_mbert_text_dataset, 
    batch_size=multilingual_mbert_perplexity_batch_size, 
    shuffle=False
)

In [ ]:
# Compute perplexity in batches
multilingual_mbert_perplexity_scores = []
for batch in tqdm(multilingual_mbert_dataloader, desc="Computing Perplexity for mBert"):
    batch_perplexities = compute_perplexity_batch(batch, multilingual_mbert_tokenizer, multilingual_mbert_model)
    multilingual_mbert_perplexity_scores.extend(batch_perplexities)

Computing Perplexity for mBert:   0%|          | 0/203447 [00:00<?, ?it/s]

W0203 15:42:02.372000 13899 site-packages/torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


In [ ]:
# Store perplexity scores in DataFrame and display
multilingual_evaluation_results_datasets["mbert"]["perplexity"] = multilingual_mbert_perplexity_scores
print(f"Perplexity Score: {multilingual_evaluation_results_datasets['mbert']['perplexity'].mean()}")

Perplexity Score: 2.2175958210491413


In [ ]:
# save perplexity
save_tmp_df(multilingual_evaluation_results_datasets["mbert"], f"multilingual_mbert_perplexity")

#### XLM-R

In [ ]:
# batch size
multilingual_xlmr_perplexity_batch_size = 8

In [ ]:
# Load tokenizers & models for both xlmr
multilingual_xlmr_tokenizer = AutoTokenizer.from_pretrained(multilingual_model_names["xlmr"])
multilingual_xlmr_model = AutoModelForMaskedLM.from_pretrained(multilingual_model_names["xlmr"]).to(device)
multilingual_xlmr_model = torch.compile(multilingual_xlmr_model)
multilingual_xlmr_model.eval()

In [ ]:
# Prepare dataset and DataLoader
multilingual_xlmr_generated_texts = multilingual_evaluation_results_datasets["xlmr"]["generated"].tolist()
multilingual_xlmr_text_dataset = TextDataset(multilingual_xlmr_generated_texts)
multilingual_xlmr_dataloader = DataLoader(
    multilingual_xlmr_text_dataset, 
    batch_size=multilingual_xlmr_perplexity_batch_size, 
    shuffle=False
)

In [ ]:
# Compute perplexity in batches
multilingual_xlmr_perplexity_scores = []
for batch in tqdm(multilingual_xlmr_dataloader, desc="Computing Perplexity for XLM-R"):
    batch_perplexities = compute_perplexity_batch(batch, multilingual_xlmr_tokenizer, multilingual_xlmr_model)
    multilingual_xlmr_perplexity_scores.extend(batch_perplexities)

In [ ]:
# Store perplexity scores in DataFrame and display
multilingual_evaluation_results_datasets["xlmr"]["perplexity"] = multilingual_xlmr_perplexity_scores
print(f"Perplexity Score: {multilingual_evaluation_results_datasets['xlmr']['perplexity'].mean()}")

In [ ]:
# save perplexity
save_tmp_df(multilingual_evaluation_results_datasets["xlmr"], f"multilingual_xlmr_perplexity")

### Save Evaluation Results

In [24]:
# combine evaluation results
for model_name in multilingual_evaluation_results_datasets.keys():
    print(f"Processing {model_name}...")

    # load metrics and set
    metrics = load_tmp_df(f"multilingual_{model_name}_metrics")
    multilingual_evaluation_results_datasets[model_name]["bleu"] = metrics["bleu"]
    multilingual_evaluation_results_datasets[model_name]["rouge-1"] = metrics["rouge-1"]
    multilingual_evaluation_results_datasets[model_name]["rouge-2"] = metrics["rouge-2"]
    multilingual_evaluation_results_datasets[model_name]["rouge-l"] = metrics["rouge-l"]
    multilingual_evaluation_results_datasets[model_name]["chrf-s"] = metrics["chrf-s"]
    multilingual_evaluation_results_datasets[model_name]["bert_score"] = metrics["bert_score"]

    # load perplexity and set
    perplexity = load_tmp_df(f"multilingual_{model_name}_perplexity")
    multilingual_evaluation_results_datasets[model_name]["perplexity"] = perplexity["perplexity"]

    save_models_df(multilingual_evaluation_results_datasets[model_name], f"multilingual_{model_name}_evaluation_results")

Processing mbert...
Processing xlmr...


In [ ]:
# display metrics
for model_name, dataset in multilingual_evaluation_results_datasets.items():
    print(f"Metrics scores for {model_name}:")
    print(f" BLEU Score: {dataset['bleu'].mean()}")
    print(f" ROUGE-1 Score: {dataset['rouge-1'].mean()}")
    print(f" ROUGE-2 Score: {dataset['rouge-2'].mean()}")
    print(f" ROUGE-L Score: {dataset['rouge-l'].mean()}")
    print(f" chrF-S Score: {dataset['chrf-s'].mean()}")
    print(f" BERT Score: {dataset['bert_score'].mean()}")
    print(f" Perplexity: {dataset['perplexity'].mean()}")

# 3. Benchmarking and Analysis
Compare the performance of LSTM BPE, LSTM Unigram, mBERT, and XLM-R using BLEU, ROUGE, and Perplexity.

## Data Preprocessing

In [ ]:
# datasets
benchmarking_models = {
    "lstm": [
        "bpe", 
        "unigram"
    ],
    "multilingual": [
        "mbert", 
        "xlmr"
    ]
}

In [ ]:
# Load and process dataset
def load_and_rename_columns_benchmarking(key, model_name):
    df = load_models_df(f"{key}_{model_name}_evaluation_results")

    # Ensure only required columns exist
    df = df[["english", "bleu", "rouge", "perplexity"]]

    column_mapping = {
        "bleu": f"bleu_{model_name}",
        "rouge": f"rouge_{model_name}",
        "perplexity": f"perplexity_{model_name}",
    }
    
    df = df.rename(columns=column_mapping)

    return df

In [ ]:
# load datasets
benchmarking_loaded_datasets = []
for key, file_list in benchmarking_models.items():
    df_list= [load_and_rename_columns_multilingual(file) for file in file_list]
    benchmarking_loaded_datasets.append(df_list)

In [ ]:
# Merge all datasets on 'english' using an outer join
benchmarking_results = benchmarking_loaded_datasets[0]
for df in benchmarking_loaded_datasets[1:]:
    benchmarking_results = benchmarking_results.merge(df, on="english", how="outer")

In [ ]:
# display merge dataset
display(benchmarking_results.head())

## Compute Average Scores for Comparison
Get mean BLEU, ROUGE, and Perplexity for LSTM (BPE & Unigram), mBERT, and XLM-R.

In [ ]:
# Define the model names and their respective column prefixes
benchmarking_model_names = ["LSTM BPE", "LSTM Unigram", "mBERT", "XLM-R"]
benchmarking_column_prefixes = ["bpe", "unigram", "mBERT", "XLM-R"]

# Compute mean scores dynamically using a dictionary comprehension
benchmarking_mean_scores = {
    model: {
        "BLEU": df[f"bleu_{prefix}"].mean(),
        "ROUGE": df[f"rouge_{prefix}"].mean(),
        "Perplexity": df[f"perplexity_{prefix}"].mean(),
    }
    for model, prefix in zip(benchmarking_model_names, benchmarking_column_prefixes)
}

# Display mean scores
pprint.pprint(benchmarking_mean_scores)

## Visualize Benchmarking Results
Plot BLEU, ROUGE, and Perplexity scores for comparison.

In [ ]:
# Plot each metric
benchmarking_metrics = ["BLEU", "ROUGE", "Perplexity"]
for metric in benchmarking_metrics:
    values = [benchmarking_mean_scores["LSTM BPE"][metric], 
              benchmarking_mean_scores["LSTM Unigram"][metric], 
              benchmarking_mean_scores["mBERT"][metric], 
              benchmarking_mean_scores["XLM-R"][metric]]

    plt.figure(figsize=(6, 4))
    plt.bar(benchmarking_model_names, values, color=["blue", "green", "orange", "red"])
    plt.title(f"{metric} Score Comparison")
    plt.xlabel("Models")
    plt.ylabel(metric)
    plt.show()

### Save Benchmarking Results

In [ ]:
# save results
save_models_df(benchmarking_results, "benchmarking_results")